### Export raw data to csv files

In [56]:
import pandas as pd
import os

def create_dataframe(directory):
    # List all parquet files in the directory that start with 'X'
    parquet_files = [f for f in os.listdir(directory) if f.startswith('X') and f.endswith('.parquet')]

    # Initialize an empty list to store dataframes
    dfs = []

    # Load and process each parquet file
    for pq_file in parquet_files:
        df = pd.read_parquet(os.path.join(directory, pq_file))
        df['data_type'] = 'train' if 'train' in pq_file else 'test'
        df['data_quality'] = 'observed' if 'observed' in pq_file else 'estimated'
        dfs.append(df)

    # Combine all dataframes
    combined_df = pd.concat(dfs, ignore_index=True)

    # Fill NULL values for 'date_calc'
    combined_df['date_calc'].fillna(method='ffill', inplace=True)

    return combined_df

# Create dataframes for 'A', 'B', and 'C' directories
df_A = create_dataframe('A')
df_B = create_dataframe('B')
df_C = create_dataframe('C')

# Display the first few rows of the dataframe for 'A' directory
df_A.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,data_type,data_quality,date_calc
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,...,285.899994,100.0,39640.101562,3.7,-3.6,-0.8,-0.0,train,observed,NaT
1,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,...,286.100006,100.0,40123.898438,3.6,-3.6,-0.6,-0.0,train,observed,NaT
2,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,...,286.299988,100.0,40628.300781,3.6,-3.6,-0.4,-0.0,train,observed,NaT
3,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,...,286.600006,100.0,41153.601562,3.5,-3.5,-0.2,-0.0,train,observed,NaT
4,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,...,286.799988,100.0,41699.898438,3.5,-3.5,0.0,-0.0,train,observed,NaT


In [57]:
def count_rows_in_parquet_files(directory):
    # List all parquet files in the directory that start with 'X'
    parquet_files = [f for f in os.listdir(directory) if f.startswith('X') and f.endswith('.parquet')]

    # Count the total number of rows in all parquet files
    total_rows = 0
    for pq_file in parquet_files:
        df = pd.read_parquet(os.path.join(directory, pq_file))
        total_rows += len(df)

    return total_rows

# Count rows in parquet files and created dataframes for 'A', 'B', and 'C' directories
rows_in_parquet_A = count_rows_in_parquet_files('A')
rows_in_parquet_B = count_rows_in_parquet_files('B')
rows_in_parquet_C = count_rows_in_parquet_files('C')

rows_in_df_A = len(df_A)
rows_in_df_B = len(df_B)
rows_in_df_C = len(df_C)

rows_in_parquet_A, rows_in_df_A, rows_in_parquet_B, rows_in_df_B, rows_in_parquet_C, rows_in_df_C

(139125, 139125, 137385, 137385, 137281, 137281)

In [58]:
# Change the datatype of 'snow_density:kgm3' to float and fill NULL for missing values
df_A['snow_density:kgm3'] = df_A['snow_density:kgm3'].astype(float).fillna('NULL')
df_B['snow_density:kgm3'] = df_B['snow_density:kgm3'].astype(float).fillna('NULL')
df_C['snow_density:kgm3'] = df_C['snow_density:kgm3'].astype(float).fillna('NULL')

# Display the datatypes of columns in the dataframe for 'A' directory
df_A.dtypes['snow_density:kgm3']

dtype('O')

In [59]:
def join_with_train_targets(directory, df):
    # Load the train_targets parquet file
    train_targets = pd.read_parquet(os.path.join(directory, 'train_targets.parquet'))

    # Join the dataframe with train_targets on the specified columns
    merged_df = df.merge(train_targets, left_on='date_forecast', right_on='time', how='left')

    # Identify rows in train_targets that do not have a matching 'date_forecast' in the dataframe
    missing_rows = train_targets[~train_targets['time'].isin(df['date_forecast'])]

    return merged_df, missing_rows

# Join dataframes with train_targets for 'A', 'B', and 'C' directories
merged_df_A, missing_rows_A = join_with_train_targets('A', df_A)
merged_df_B, missing_rows_B = join_with_train_targets('B', df_B)
merged_df_C, missing_rows_C = join_with_train_targets('C', df_C)

# Display the number of missing rows for each directory
len(missing_rows_A), len(missing_rows_B), len(missing_rows_C)

(24, 25, 25)

In [60]:
# Add the missing rows to the corresponding dataframes and fill None for the other columns
def add_missing_rows(df, missing_rows):
    # Create a new dataframe with the same columns as the original dataframe
    new_df = pd.DataFrame(columns=df.columns)

    # Fill the 'date_forecast' column with the 'time' values from the missing rows
    new_df['date_forecast'] = missing_rows['time']

    # Fill the 'pv_measurement' column
    new_df['pv_measurement'] = missing_rows['pv_measurement']

    # Append the new dataframe to the original dataframe
    updated_df = pd.concat([df, new_df], ignore_index=True)

    return updated_df

# Add missing rows to the dataframes for 'A', 'B', and 'C' directories
updated_df_A = add_missing_rows(merged_df_A, missing_rows_A)
updated_df_B = add_missing_rows(merged_df_B, missing_rows_B)
updated_df_C = add_missing_rows(merged_df_C, missing_rows_C)

# Display the last few rows of the updated dataframe for 'A' directory to confirm the addition
updated_df_A.tail(10)

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,data_type,data_quality,date_calc,time,pv_measurement
139139,2023-01-26 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,58.08
139140,2023-01-26 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139141,2023-01-26 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139142,2023-01-26 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139143,2023-01-26 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139144,2023-01-26 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139145,2023-01-26 20:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139146,2023-01-26 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139147,2023-01-26 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00
139148,2023-01-26 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,0.00


In [61]:
# Correct the datatype of 'snow_density:kgm3' such that available numbers are float and missing values are None
def correct_snow_density_dtype(df):
    df['snow_density:kgm3'] = df['snow_density:kgm3'].replace('NULL', None).astype(float)
    return df

# Apply the correction to the dataframes for 'A', 'B', and 'C' directories
corrected_df_A = correct_snow_density_dtype(updated_df_A)
corrected_df_B = correct_snow_density_dtype(updated_df_B)
corrected_df_C = correct_snow_density_dtype(updated_df_C)

# Display the datatypes of columns in the corrected dataframe for 'A' directory
corrected_df_A.dtypes['snow_density:kgm3']

dtype('float64')

In [62]:
# Sort each dataframe by 'date_forecast' in ascending order
sorted_df_A = corrected_df_A.sort_values(by='date_forecast', ascending=True)
sorted_df_B = corrected_df_B.sort_values(by='date_forecast', ascending=True)
sorted_df_C = corrected_df_C.sort_values(by='date_forecast', ascending=True)

# Display the first few rows of the sorted dataframe for 'A' directory to confirm the sorting
sorted_df_A.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,data_type,data_quality,date_calc,time,pv_measurement
0,2019-06-02 22:00:00,7.7,1.230,1744.900024,0.0,0.0,1744.900024,0.0,280.299988,0.0,...,39640.101562,3.7,-3.6,-0.8,-0.0,train,observed,NaT,2019-06-02 22:00:00,0.0
1,2019-06-02 22:15:00,7.7,1.229,1734.000000,0.0,0.0,1734.000000,0.0,280.299988,0.0,...,40123.898438,3.6,-3.6,-0.6,-0.0,train,observed,NaT,NaT,NaN
2,2019-06-02 22:30:00,7.7,1.228,1723.500000,0.0,0.0,1723.500000,0.0,280.299988,0.0,...,40628.300781,3.6,-3.6,-0.4,-0.0,train,observed,NaT,NaT,NaN
3,2019-06-02 22:45:00,7.7,1.226,1713.400024,0.0,0.0,1713.400024,0.0,280.299988,0.0,...,41153.601562,3.5,-3.5,-0.2,-0.0,train,observed,NaT,NaT,NaN
4,2019-06-02 23:00:00,7.7,1.225,1703.599976,0.0,0.0,1703.599976,0.0,280.299988,0.0,...,41699.898438,3.5,-3.5,0.0,-0.0,train,observed,NaT,2019-06-02 23:00:00,0.0


In [63]:
# Delete 'time' column
sorted_df_A = sorted_df_A.drop(columns=['time'])
sorted_df_B = sorted_df_B.drop(columns=['time'])
sorted_df_C = sorted_df_C.drop(columns=['time'])

In [64]:
# Export the sorted dataframes to CSV files
sorted_df_A.to_csv('csv_files/A_raw.csv', index=False)
sorted_df_B.to_csv('csv_files/B_raw.csv', index=False)
sorted_df_C.to_csv('csv_files/C_raw.csv', index=False)

In [65]:
# Add a column to each dataframe indicating its source (A, B, or C)
sorted_df_A['site'] = 'A'
sorted_df_B['site'] = 'B'
sorted_df_C['site'] = 'C'

# Concatenate the dataframes
combined_df = pd.concat([sorted_df_A, sorted_df_B, sorted_df_C], ignore_index=True)

# Sort the combined dataframe by 'date_forecast' and 'site' in ascending order
sorted_combined_df = combined_df.sort_values(by=['date_forecast', 'site'], ascending=[True, True])

# Display the first few rows of the sorted combined dataframe
sorted_combined_df.head()

,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,diffuse_rad:W,...,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms,data_type,data_quality,date_calc,pv_measurement,site
139149,2018-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,0.0,B
276559,2018-12-31 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,C
139150,2019-01-01 00:00:00,5.6,1.239,1226.400024,0.0,0.0,1226.400024,0.0,275.399994,0.0,...,19275.900391,2.5,1.1,2.2,-0.0,train,observed,NaT,0.0,B
276560,2019-01-01 00:00:00,5.6,1.240,1153.599976,0.0,0.0,1153.599976,1.0,275.200012,0.0,...,18220.400391,2.5,1.1,2.2,-0.0,train,observed,NaT,NaN,C
139151,2019-01-01 00:15:00,5.6,1.239,1209.300049,0.0,0.0,1209.300049,0.0,275.200012,0.0,...,17638.000000,2.7,1.2,2.4,-0.0,train,observed,NaT,NaN,B


In [66]:
# Export combined data
sorted_combined_df.to_csv('csv_files/All_raw.csv', index=False)

Note: Manually delete the first useless row in B, C and ALL